# Look at the results

## Imports

In [ ]:
import os
import torch
import matplotlib.pyplot as plt

In [ ]:
from trustgan.training import Training

## Constants

In [ ]:
path2dataset = "$HOME/Workspace/mlsecconfops/trustgan/xps/data/MNIST"
nb_classes = 10
batch_size = 32
network_name = "Net"
device = "cuda:2"
verbose = True

In [ ]:
path_to_load_net = "$HOME/Workspace/mlsecconfops/trustgan/xps/mnist-wo-gan/nets/net-best-valid-loss.pth"
path_to_load_gan = "$HOME/Workspace/mlsecconfops/trustgan/xps/mnist-wo-gan/gan.pth"

## Loot at results

In [ ]:
path2dataset = os.path.expandvars(path2dataset)
path_to_load_net = os.path.expandvars(path_to_load_net)
path_to_load_gan = os.path.expandvars(path_to_load_gan)

In [ ]:
training_pipeline = Training(
    path_to_save=None,
    path2dataset=path2dataset,
    nb_classes=nb_classes,
    num_epochs=None,
    nb_step_net_gan=None,
    nb_step_gan=None,
    nb_step_net_alone=None,
    prop_net_alone=None,
    path_to_load_net=path_to_load_net,
    path_to_load_gan=path_to_load_gan,
    batch_size=batch_size,
    device_name=device,
    verbose=verbose,
    path2net=None,
    network_name=network_name,
)

In [ ]:
training_pipeline.epoch = None

In [ ]:
# training_pipeline.get_example(training_pipeline.validloader)

In [ ]:
training_pipeline.net.eval()

inputs, labels = next(iter(training_pipeline.testloader))
inputs, labels = inputs.to(training_pipeline.device), labels.to(
    training_pipeline.device
)
inputs, labels = training_pipeline.modifier((inputs, labels))
_, labels = torch.max(labels, 1)

net_outputs = training_pipeline.net(inputs)

net_outputs = torch.nn.functional.softmax(net_outputs, dim=1)
score_pred, predicted = torch.max(net_outputs, 1)

# if score_pred.ndim > 1:
#    score_pred = score_pred.mean(axis=tuple(range(1, score_pred.ndim)))
#    predicted = predicted.to(torch.float).mean(
#        axis=tuple(range(1, predicted.ndim))
#    )
#    labels = labels.to(torch.float).mean(axis=tuple(range(1, labels.ndim)))
#
# idx_min = torch.argmin(score_pred)
# idx_max = torch.argmax(score_pred)
#
# for idx, name in [(idx_min, "min"), (idx_max, "max")]:
#
#    images = inputs[idx].cpu().detach().numpy()
#
# training_pipeline.net.train()

In [ ]:
for j in [1]:
    X = inputs.detach().cpu()
    y = labels.detach().cpu()
    y_hat = predicted.detach().cpu()
    y_conf = score_pred.detach().cpu()

    el = torch.cat([X[j, i, ...][..., None] for i in range(X.shape[1])], axis=-1)
    plt.imshow(el)
    if y.ndim == 1:
        plt.title(int(y[[j]]))
    plt.show()

    if y.ndim == 3:
        plt.imshow(y[j])
        plt.show()

    if y_hat.ndim == 3:
        plt.imshow(y_hat[j])
        plt.show()

    if y_conf.ndim == 3:
        plt.imshow(y_conf[j])
        plt.colorbar()
        plt.show()